In [19]:
import networkx as nx

G = nx.adjacency_matrix(nx.read_graphml("graphs/4_500.graphml")).todense()
G

array([[ 0, 12, 10,  0],
       [12,  0,  0, 16],
       [10,  0,  0,  0],
       [ 0, 16,  0,  0]])

In [20]:
def calculate_cut_weight(graph, S):
    """
    complexidade k(n-k), k = len(subset)
    W(n^2) e B(0)
    O(n^2)

    input: [[matriz adjacencia]], {subset1}, {subset2}
    """
    total_weight = 0
    for u in S:
        for v in set(range(len(graph))) - S:
            total_weight += graph[u][v]
    
    return total_weight

---

# exhaustive search

- ~~recurrence em q  - return max({A...}{...D} , {A...D}{...})~~

- iterative - compare {A}{...} {AB}{...} ... {A...}{Z}

penso q ambos seriam 2^n de complexidade pq arvore e n+n-1+n-2+... ?

In [21]:
import itertools

def exhaustive_search(graph):
    input_set = set(range(len(graph)))
    subsets = []
    n = len(input_set)
    
    # generate all subsets (complexy 2^V to generate * V to convert to set = O(V^2 * V))
    for r in range(n + 1):
        for subset in itertools.combinations(input_set, r):
            subsets.append(set(subset))

    best = input_set
    weight = 0
    for subset in subsets: # 2^n resultados para percorrer
        new_weight = calculate_cut_weight(graph, subset) # k*(n-k) q é < 2^n , ent O(2^n * n^2)
        if new_weight > weight:
            best = subset
            weight = new_weight
        else:
            pass
    
    return best, input_set-best, weight

exhaustive_search(G)

({0, 3}, {1, 2}, np.int64(38))

---

# greedy heuristic

pode ser sorted weights e maior é AB, ent S = {A} T = {B}, segudo maior CD, ent S={AC} T={BD}, algo desse genero

mas se segundo maior é AC - S = {A} T = {BC} :: 

- if A and C in S or T: pass

- if A in S or T: C to other

- if neither in: randomly select - could compare to already in vertices but to complex?

complexidade nlogn por causa do sort + n por iterar por cada um - logo complexidade final = nlogn ?

In [22]:
def max_weighted_cut_greedy(G):
    num_vertices = len(G)
    
    # Step 1: Extract edges and their weights
    edges = []
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):  # To avoid duplicate edges
            weight = G[i, j]
            edges.append((i, j, weight))

    # Step 2: Sort edges in descending order based on their weights
    edges.sort(key=lambda e: e[2], reverse=True)
    
    seen, S, T = set(), set(), set()

    # Step 3: Process each edge
    for u, v, weight in edges:
        if u not in seen and v not in seen: 
            # nenhum vertice visto
            seen.update({u,v})
            S.add(u)
            T.add(v)
        elif u in S and v not in seen:
            # u no primeiro set, v não visto
            seen.add(v)
            T.add(v)
        elif u in T and v not in seen:
            # u no segundo set, v não visto
            seen.add(v)
            S.add(v)
        elif v in S and u not in seen:
            # v no primeiro set, u não visto
            seen.add(u)
            T.add(u)
        elif v in T and u not in seen:
            # v no segundo set, u não visto
            seen.add(u)
            S.add(u)
        else:
            #u e v vistos
            pass

    return S, T, calculate_cut_weight(G, S)


# Run the greedy heuristic
S, T, weight = max_weighted_cut_greedy(G)

print("Subset S:", S)
print("Subset T:", T)
print("Weight:", weight)


Subset S: {1, 2}
Subset T: {0, 3}
Weight: 38


---

In [23]:
import os

graphs = sorted([f"graphs/{x}" for x in os.listdir("graphs") if x[-7:] == "graphml"])[:20] # 188

for G in graphs:
    G = nx.adjacency_matrix(nx.read_graphml(G)).todense()
    s,t, w = exhaustive_search(G)
    s1,t1,w1 = max_weighted_cut_greedy(G)
    print(len(G), end = ": ")
    print(round(w1/w,2))

10: 1.0
10: 0.88
10: 1.0
10: 0.87
11: 1.0
11: 1.0
11: 0.87
11: 0.91
12: 1.0
12: 1.0
12: 0.93
12: 0.84
13: 0.98
13: 0.83
13: 0.87
13: 0.85
14: 0.98
14: 0.87
14: 0.94
14: 0.9
